In [6]:
! pip install openpyxl gensim fasttext lightgbm multipledispatch razdel nmslib

In [18]:
import pandas as pd
import os

path = "/kaggle/input/hacksai-3/"
temp = "/kaggle/working/"

In [43]:
file = path + 'outlier_whole_preds.csv'
df = pd.read_csv(file, sep=';')

# Запоминаем уникальные строки
#start_id = DataSet.objects.all().order_by("-id_without_explode").first()
start_id = None
if not start_id:
    start_id = 0
else:
    start_id = start_id.id_without_explode
df.drop_duplicates(inplace=True)
df['id_without_explode'] = range(start_id + 1, start_id + len(df) + 1)

df.rename(columns={
    "Номер продукции": "product_number",
    "Коды ТН ВЭД ЕАЭС": "ved_code_id",
    "Технические регламенты": 'technical_regulations',
    "Группа продукции": 'product_group',
    "Общее наименование продукции": 'product_name',
    "ИЛ": "lab_name",
    "Заявитель": 'requester_name',
    "Адрес Заявителя": 'requester_address',
    "Изготовитель": 'producer_name',
    "Страна": 'producer_country',
    "Адрес изготовителя": 'producer_address'
}, inplace=True)

df['requester_zipcode'] = df['requester_address'].str.split() \
    .apply(lambda x: x[1] if len(x) > 1 else None)

df['producer_zipcode'] = df['producer_address'].str.split() \
    .apply(lambda x: x[1] if len(x) > 1 else None)

df['producer_country_code'] = df['producer_address'].str.split().apply(lambda x: x[0].replace('РОССИЯ', 'RU'))
df['requester_country_code'] = df['requester_address'].str.split().apply(lambda x: x[0].replace('РОССИЯ', 'RU'))

In [31]:
df.head()

In [44]:
df['product_group'] = df['product_group'].fillna('').astype(str).str.split("; ").apply(lambda x: list(set(x)))
df = df.explode(column='product_group')
df['ved_code_id'] = df['ved_code_id'].fillna('').astype(str).str.split("; ").apply(lambda x: list(set(x)))
df = df.explode(column='ved_code_id')
df['technical_regulations'] = df['technical_regulations'].fillna('').astype(str).str.split("; ").apply(lambda x: list(set(x)))
df = df.explode(column='technical_regulations')

start_id = None#start_id = DataSet.objects.all().order_by("-id").first()
if not start_id:
    start_id = 0
else:
    start_id = start_id.id
    
df.drop_duplicates(inplace=True)

df['id'] = range(start_id + 1, start_id + len(df) + 1)


In [73]:
zip_codes = pd.DataFrame()

for file in os.listdir(path+'csv/'):
    zip_code = pd.read_csv(path+'csv/'+file, dtype={'zipcode':str})
    zip_codes = pd.concat([zip_codes, zip_code])

zip_codes = zip_codes[['country_code', 'zipcode', 'latitude', 'longitude']].drop_duplicates(subset=['country_code', 'zipcode'])

df = pd.merge(
             pd.merge(df, zip_codes.rename(columns={'zipcode':'requester_zipcode',
                                           'country_code':'requester_country_code'
                                      }), how='left'),
        zip_codes.rename(columns={'zipcode':'producer_zipcode',
                              'country_code':'producer_country_code'
                                      }), how='left')

In [98]:
df['GRUPPA'] = df['ved_code_id'].apply(lambda x: x[:2])
df['TOV_POZ'] = df['ved_code_id'].apply(lambda x: x[2:4])
df['SUB_POZ'] = df['ved_code_id'].apply(lambda x: x[4:])

ved_dict = pd.read_csv(path+'ved_dict.csv', sep=';', dtype=str).dropna().drop_duplicates()
gruppa_dict = dict(ved_dict[['GRUPPA', 'NAIM2']].values)
tov_poz_dict = dict(ved_dict[['TOV_POZ', 'NAIM3']].values)
sub_poz_dict = dict(ved_dict[['SUB_POZ', 'NAIM4']].values)

df['NAIM2'] = df['GRUPPA'].map(gruppa_dict)
df['NAIM3'] = df['TOV_POZ'].map(tov_poz_dict)
df['NAIM4'] = df['SUB_POZ'].map(sub_poz_dict)

In [101]:
df.shape

In [102]:
df.to_csv('db_file.csv', sep=';')